# Pre-Processing

1. Epochs for selected region (window=5s)
2. Transform raw data to frequency domain, look for peaks at 0.4-5 Hz
3. outliners identification -- e.g. 95% percentile
4. Filter out epochs based on:
    - peaks at 0.4-5 Hz
    - within 95% percentile of power? 


# Manifold Construction